In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
import cv2,os
import tkinter as tk
from tkinter import filedialog

In [3]:
from tensorflow.keras.models import load_model
import numpy as np

### Data preprocessing

In [4]:


dataset = "dataset"
labels_dict = {"with mask":0,"without mask":1}
data = []
target = []
categories = ['with mask','without mask']
for category in categories:
    folder = os.path.join(dataset,category)
    img_dir = os.listdir(folder)

    for image in img_dir:    
        im = os.path.join(folder,image)
        img = cv2.imread(im)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        resized = cv2.resize(img,(100,100))
        data.append(resized)
        target.append(labels_dict[category])

normalized_data = np.array(data)/255.0
reshaped_data = np.reshape(normalized_data,(normalized_data.shape[0],100,100,3))
target = np.array(target)

from tensorflow.keras.utils import to_categorical

categorical_target = to_categorical(target)

np.save("data",reshaped_data)
np.save("target",categorical_target)

        

### Model training

In [13]:
data = np.load("data.npy")
target = np.load("target.npy")

model = Sequential()

# *** layers ***

model.add(Conv2D(200,(3,3), input_shape = data.shape[1:] )) ## CNN Layer 
model.add(Activation("relu"))                               ## "RELU" Activation Function
model.add(MaxPooling2D(pool_size=(2,2)))                    ## MaxPooling layers

model.add(Conv2D(100,(3,3))) ## CNN Layer 
model.add(Activation("relu"))                               ## "RELU" Activation Function
model.add(MaxPooling2D(pool_size=(2,2)))                    ## MaxPooling layers

model.add(Flatten()) ## flattening 
model.add(Dropout(0.5)) ## To reduce Overfitting

model.add(Dense(50,activation="relu")) 
model.add(Dense(2,activation="softmax")) ## FINAL LAYER

model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

train_X,test_X,train_y,test_y = train_test_split(data,target,test_size=0.1)

checkpoint = ModelCheckpoint('model-ready.model',monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

model.fit(train_X,train_y,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
31/31 [==============================] - ETA: 0s - loss: 0.5022 - accuracy: 0.7485WARNING:tensorflow:From D:\Anaconda\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model-ready.model\assets
31/31 [==============================] - 50s 2s/step - loss: 0.5022 - accuracy: 0.7485 - val_loss: 0.1798 - val_accuracy: 0.9637
Epoch 2/20
31/31 [==============================] - 50s 2s/step - loss: 0.1728 - accuracy: 0.9313 - val_loss: 0.1537 - val_accuracy: 0.9516
Epoch 3/20
31/31 [==============================] - 58s 2s/step - loss: 0.1289 - accuracy: 0.9525 - val_loss: 0.1413 - val_accuracy: 0.9556
Epoch 4/20
31/31 [==============================] - 61s 2s/step - loss: 0.1004 - accuracy:

In [14]:
print(model.evaluate(test_X,test_y))

5/5 [==============================] - 1s 160ms/step - loss: 0.0930 - accuracy: 0.9710
[0.09304454922676086, 0.9710144996643066]


### Computer Vision and File Upload

In [25]:
model = load_model("model-ready.model")
root = tk.Tk()
root.withdraw()

upload = filedialog.askopenfilename()


labels = {0:"Mask",1:"No Mask"}

color_dict = {0: (0,255,0),1:(0,0,255)}

img = cv2.imread(upload)


rgbimg = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

resized = cv2.resize(rgbimg,(100,100))
normalize = resized/255.0
normalize.shape
reshaped = np.reshape(normalize,(1,100,100,3))

result = model.predict(reshaped)
print(result)
label = np.argmax(result,axis=1)[0]
print(label)

image = cv2.rectangle(img, (0,0), (150,80), color_dict[label], 2)
image = cv2.putText(image,labels[label],(30,50),cv2.FONT_HERSHEY_SIMPLEX,0.8,color_dict[label])
cv2.imshow("image",image)
    
cv2.waitKey(0)
cv2.destroyAllWindows()

[[0.02979906 0.97020096]]
1
